# Lecture 12. Intro to Deep Learning with pytorch

[Pytorch site](https://pytorch.org/)

[An actually excellent kaggle tutorial](https://www.kaggle.com/code/alejopaullier/introduction-to-lstm)

[Installation instructions:](https://pytorch.org/get-started/locally/) Read carefully.  
Choose the right installation for your system.   

If you want, make in a different folder, install conda and create a new venv for pytorch using conda or just ```pip install torch```.  
I recommend to use a separate folder and install conda to create a different virtual enviroment,

## Definitions

### tensor  
### layers

In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder 
# use this for categorical nominal (not Ordinal )target variables

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


In [2]:
# Load the CSV data into a pandas DataFrame
df = pd.read_csv('har_train.csv')

# Convert the DataFrame into a PyTorch tensor

## EDA
[kaggle notebook](https://www.kaggle.com/code/abheeshthmishra/eda-of-human-activity-recognition)

## Data preprocessing

In [3]:
df.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('O')], dtype=object)

In [4]:
# Select columns with dtype('O') Object
object_columns = df.select_dtypes(include=['O']).columns

print(object_columns)

Index(['Activity'], dtype='object')


In [5]:
# target variable to encode
target = df['Activity']

# Initialize a label encoder
encoder = LabelEncoder()

# Fit and transform the target column
df['Activity'] = encoder.fit_transform(target)
df.Activity.head()

0    2
1    2
2    2
3    2
4    2
Name: Activity, dtype: int64

In [6]:
df.dtypes.unique()

array([dtype('float64'), dtype('int64')], dtype=object)

In [7]:
# tensor = torch.tensor(df.values.astype(float))  #
tensor_hra = torch.tensor(df.values).float()

tensor_hra

tensor([[ 2.8858e-01, -2.0294e-02, -1.3291e-01,  ..., -5.8627e-02,
          1.0000e+00,  2.0000e+00],
        [ 2.7842e-01, -1.6411e-02, -1.2352e-01,  ..., -5.4317e-02,
          1.0000e+00,  2.0000e+00],
        [ 2.7965e-01, -1.9467e-02, -1.1346e-01,  ..., -4.9118e-02,
          1.0000e+00,  2.0000e+00],
        ...,
        [ 2.7339e-01, -1.7011e-02, -4.5022e-02,  ...,  4.0811e-02,
          3.0000e+01,  5.0000e+00],
        [ 2.8965e-01, -1.8843e-02, -1.5828e-01,  ...,  2.5339e-02,
          3.0000e+01,  5.0000e+00],
        [ 3.5150e-01, -1.2423e-02, -2.0387e-01,  ...,  3.6695e-02,
          3.0000e+01,  5.0000e+00]])

In [8]:
features_tensor = tensor_hra[:, :-1]
target_tensor = tensor_hra[:, -1]
target_tensor

tensor([2., 2., 2.,  ..., 5., 5., 5.])

In [9]:
#tensor_hra is your input data and target_tensor is your target data
dataset = TensorDataset(features_tensor, target_tensor)

inputs = features_tensor
# Define your batch size
batch_size = inputs.size(0)  
# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

NUM_FEATURES: This is the number of features in your data, which corresponds to the second dimension of your data tensor.
HIDDEN_SIZE: This is a hyperparameter that you choose. It represents the number of features in the hidden state. In your case, you've chosen 16.
NUM_LAYERS: This is also a hyperparameter that you choose. It represents the number of LSTM layers in the LSTM network. In your case, you've chosen 2.
NUM_CLASSES: This is the number of unique values in your target variable. You can get this by applying the torch.unique() function to your target tensor and getting its length.

In [10]:
NUM_FEATURES = features_tensor.shape[1]  # Number of features in your data

In [11]:
NUM_FEATURES

562

In [12]:
inputs.numel()

4131824

In [ ]:
print(f"Inputs shape: {inputs.shape}")

In [ ]:
# import a ready lstm model for the data
# Define the LSTM model


In [27]:
# Initial project configuration
result = []
project_name = 'Human Activity Recognition'
arch = "Convolution + pooling + convolution + pooling + dense + dense + dense + output"
batch_size = 64
epochs = 50
lr = 0.01
momentum = 0.9
torch.manual_seed(29)

In [48]:
BATCH_SIZE = 64  # Define your batch size
# SEQUENCE_LENGTH = 50  # Define your sequence length
INPUT_SIZE = 562  # Define your input size
HIDDEN_SIZE = 16  # Define the number of features in the hidden state
NUM_LAYERS = 10

In [45]:
# Assuming 'dataset' is your Dataset object

dataset = TensorDataset(features_tensor, target_tensor)

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [46]:
# === LSTM Network ===
LSTM = nn.LSTM(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,        
    num_layers=NUM_LAYERS,       
    batch_first=True
)


In [47]:


# Now, you can iterate over the DataLoader object to get your inputs
for i, (inputs, labels) in enumerate(dataloader):
    # Ensure inputs is of the correct shape (BATCH_SIZE, INPUT_SIZE)
    inputs = inputs.view(BATCH_SIZE, INPUT_SIZE)

    # Initialize hidden and cell states
    h0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE)
    c0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE)

    # Pass inputs and initial states through LSTM
    outputs, (hn, cn) = LSTM(inputs, (h0, c0))

    print(f"Input batch shape: {inputs.shape}")
    print(f"Hidden state 0 shape: {h0.shape}")
    print(f"Cell state 0 shape: {c0.shape}")
    print(f"Output batch shape: {outputs.shape}")

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [12]:
HIDDEN_SIZE = 16  # Number of features in the hidden state
NUM_LAYERS = 2  # Number of LSTM layers in the LSTM network
NUM_CLASSES = len(torch.unique(target_tensor))

In [13]:
class ClassificationModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ClassificationModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.layer3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return F.log_softmax(x, dim=1)

In [14]:
model = ClassificationModel(input_size=562, hidden_size=20, num_classes=NUM_CLASSES)

In [22]:
# Perform a forward pass
outputs = model(inputs)

In [23]:
labels=target_tensor.long()

In [24]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
labels = labels.long()
loss = criterion(outputs, labels)

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
epochs = 5

In [ ]:


# === LSTM Network ===
LSTM = nn.LSTM(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,        
    num_layers=NUM_LAYERS,       
    batch_first=True
)

inputs = torch.randn(BATCH_SIZE,  INPUT_SIZE) # LSTM Network input
h0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE) # Initial Hidden state
c0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE) # Initial Cell state
outputs, (hn, cn) = LSTM(inputs, (h0, c0)) # LSTM Network output

print(f"Input batch shape: {inputs.shape}")
print(f"Hidden state 0 shape: {h0.shape}")
print(f"Cell state 0 shape: {c0.shape}")
print(f"Output batch shape: {outputs.shape}")

In [ ]:
BATCH_SIZE = 8 # Number of sequences in your batch
HIDDEN_SIZE = 16 # Number of features in the hidden state 
INPUT_SIZE = 563 # Size of each embedding in the sequence
NUM_LAYERS = 3 # Number of LSTM layers in the LSTM network
SEQUENCE_LENGTH = 50 # Number of embeddings per sequence

# === LSTM Network ===
LSTM = nn.LSTM(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,        
    num_layers=NUM_LAYERS,       
    batch_first=True
)

inputs = tensor_hra
# h0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE) # Initial Hidden state
# c0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE) # Initial Cell state
outputs= LSTM(inputs) # LSTM Network output

print(f"Input batch shape: {inputs.shape}")
print(f"Hidden state 0 shape: {h0.shape}")
print(f"Cell state 0 shape: {c0.shape}")
# print(f"Output batch shape: {outputs.shape}")

In [ ]:
class Net(nn.Module):
    def __init__(self, tensor_data):
        super(Net, self).__init__()
        self.tensor_data = tensor_data
        self.fc1 = nn.Linear(16, 32)  # 16 input units to 32 output units
        self.fc2 = nn.Linear(32, 64)  # 32 input units to 64 output units
        self.fc3 = nn.Linear(64, 10)  # 64 input units to 10 output units

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create the network
net = Net(tensor_hra)
print(net)


# Perform a forward pass
out = net()
print(out)

In [ ]:
# Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(16, 32)  # 16 input units to 32 output units
        self.fc2 = nn.Linear(32, 64)  # 32 input units to 64 output units
        self.fc3 = nn.Linear(64, 10)  # 64 input units to 10 output units

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create the network
net = Net()
print(net)

# Create a random tensor of size (1, 16)
input = torch.randn(1, 16)

# Perform a forward pass
out = net(input)
print(out)

In [ ]:

# Define the network using nn.Sequential
net = nn.Sequential(
    nn.Linear(16, 32),  # 16 input units to 32 output units
    nn.ReLU(),
    nn.Linear(32, 64),  # 32 input units to 64 output units
    nn.ReLU(),
    nn.Linear(64, 10)   # 64 input units to 10 output units
)

print(net)




# Perform a forward pass
out = net(input)
print(out)

In [ ]:
BATCH_SIZE = 8 # Number of sequences in your batch
HIDDEN_SIZE = 16 # Number of features in the hidden state 
INPUT_SIZE = 384 # Size of each embedding in the sequence
NUM_LAYERS = 2 # Number of LSTM layers in the LSTM network
SEQUENCE_LENGTH = 50 # Number of embeddings per sequence

# === LSTM Network ===
LSTM = nn.LSTM(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,        
    num_layers=NUM_LAYERS,       
    batch_first=True
)

inputs = torch.randn(BATCH_SIZE, SEQUENCE_LENGTH, INPUT_SIZE) # LSTM Network input
h0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE) # Initial Hidden state
c0 = torch.randn(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE) # Initial Cell state
outputs, (hn, cn) = LSTM(inputs, (h0, c0)) # LSTM Network output

print(f"Input batch shape: {inputs.shape}")
print(f"Hidden state 0 shape: {h0.shape}")
print(f"Cell state 0 shape: {c0.shape}")
print(f"Output batch shape: {outputs.shape}")